In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
529,"Having not read the book, I was more open to t...",positive
118,I highly reccommend this movie. It blurs the l...,positive
904,THE ATTIC starts off well. The somewhat dreary...,negative
548,Highly flawed but just about watchable `comedy...,negative
831,Hard to believe this was directed by Fritz Lan...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
529,read book open fresh interpretation director g...,positive
118,highly reccommend movie blur line childhood fa...,positive
904,attic start well somewhat dreary story helped ...,negative
548,highly flawed watchable comedy run like farrel...,negative
831,hard believe directed fritz lang since mostly ...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
529,read book open fresh interpretation director g...,1
118,highly reccommend movie blur line childhood fa...,1
904,attic start well somewhat dreary story helped ...,0
548,highly flawed watchable comedy run like farrel...,0
831,hard believe directed fritz lang since mostly ...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=50) # top 50 most frequent words
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/durvankur/MLOps_Capstone_project.mlflow')
dagshub.init(repo_owner='durvankur', repo_name='MLOps_Capstone_project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-12-31 18:22:20,009 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/durvankur/MLOps_Capstone_project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "durvankur/MLOps_Capstone_project"

2025-12-31 18:22:20,048 - INFO - Initialized MLflow to track repo "durvankur/MLOps_Capstone_project"


Repository durvankur/MLOps_Capstone_project initialized!

2025-12-31 18:22:20,056 - INFO - Repository durvankur/MLOps_Capstone_project initialized!


<Experiment: artifact_location='mlflow-artifacts:/2c32229235ae4a64ad9bb18a38f3cbce', creation_time=1767184955386, experiment_id='0', last_update_time=1767184955386, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-31 18:22:21,444 - INFO - Starting MLflow run...
2025-12-31 18:22:22,001 - INFO - Logging preprocessing parameters...
2025-12-31 18:22:23,524 - INFO - Initializing Logistic Regression model...
2025-12-31 18:22:23,529 - INFO - Fitting the model...
2025-12-31 18:22:23,631 - INFO - Model training complete.
2025-12-31 18:22:23,634 - INFO - Logging model parameters...
2025-12-31 18:22:24,023 - INFO - Making predictions...
2025-12-31 18:22:24,024 - INFO - Calculating evaluation metrics...
2025-12-31 18:22:24,051 - INFO - Logging evaluation metrics...
2025-12-31 18:22:25,961 - INFO - Saving and logging the model...
2025/12/31 18:22:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-31 18:22:48,805 - INFO - Model training and logging completed in 26.80 seconds.
2025-12-31 18:22:48,807 - INFO - Accuracy: 0.71
2025-12-31 18:22:48,810 - INFO - Precision: 0.7307692307692307
2025-12-31 18:22:48,812 - INFO - Recall: 0.7169811320754716
2025-12-31 

🏃 View run silent-finch-50 at: https://dagshub.com/durvankur/MLOps_Capstone_project.mlflow/#/experiments/0/runs/6033f2f0f0f64605ab9a2ad51cf83fbb
🧪 View experiment at: https://dagshub.com/durvankur/MLOps_Capstone_project.mlflow/#/experiments/0
